In [2]:
import requests
import requests
from datetime import datetime
import pandas as pd


### Работа с ссылками


In [175]:
DOMAIN = VK_LINK
DOMAIN = DOMAIN.replace("https://vk.com/", "")
name = NAME
path = rf'{PATH}\{name}.csv'
path_com = rf'{PATH_COM}\{name}.csv'
path_users = rf'{PATH_USERS}\{name}.csv'

mossobyanin


In [176]:
TOKEN_USER = ''
VERSION = '5.84'
data = []
offset = 0
for i in range(50):
        try:
                response = requests.get('https://api.vk.com/method/wall.get',
                params={'access_token': TOKEN_USER,
                'v': VERSION,
                'domain': DOMAIN,
                'offset': offset,
                'count': 100,
                'filter': str('owner')})

                data1 = response.json()['response']['items']
                for elem in data1:
                        data.append(elem)
                offset += 100
        except Exception as e:
                print(e)
                break

In [177]:
# создание pd.DF с данными постов
df = pd.DataFrame(columns = ['date', 'owner_id', 'likes.count', 'comments.count', 'reposts.count', 'views.count', 'text'])
id = ''
for elem in data:
    id_1 = elem['id']
    df.loc[id_1, 'owner_id'] = elem['owner_id']
    df.loc[id_1, 'likes.count'] = elem['likes']['count']
    df.loc[id_1, 'comments.count']= elem['comments']['count']
    df.loc[id_1, 'reposts.count'] = elem['reposts']['count']
    try:
        df.loc[id_1, 'views.count'] = elem['views']['count']
    except:
        continue
    df.loc[id_1, 'text'] = elem['text'] = elem['text']
    df.loc[id_1, 'date'] = datetime.fromtimestamp(elem['date'])   

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
comments = pd.DataFrame(columns = ['date', 'post_id', 'sender_id', 'text'])
for i in range(5000):
    if i % 850 == 0: 
        print(i)
    if len(comments) > 15000:
        break
    if len(comments) % 5000 == 0:
        print(len(comments))
    try:
        post_id = str(df['index'][i])
        owner_id = df['owner_id'][i]
        
        response = requests.get('https://api.vk.com/method/wall.getComments',
            params={'access_token': TOKEN_USER,
            'v': VERSION,
            'owner_id' : owner_id,
            'post_id' : post_id,
            'start_comment_id': 0,
            'offset': 0,
            'count': 100,
            'sort': 'asc'})
        data2 = response.json()['response']['items']

        for elem in data2:
            id_2 = elem['id']
            comments.loc[id_2, 'text'] = elem['text']
            comments.loc[id_2, 'sender_id'] = elem['from_id']
            comments.loc[id_2, 'date'] = datetime.fromtimestamp(elem['date'])
            comments.loc[id_2, 'post_id'] = post_id
    except:
        continue


In [182]:
comments = comments.reset_index()

In [ ]:
comments = comments.sort_values(by=['date', 'post_id'])

#### Обработка информации о пользователях

In [ ]:
people = pd.DataFrame(columns = ['id', 'city', 'sex', 'first_name', 'last_name', 'university', 'faculty', 'career', 'b_date', 'relation'])

In [ ]:
fields = 'deactivated,is_closed,about,b_date,career,city,relation,universities,education,sex'
response_error_counter = 0
for i in range(len(comments)):
    user_id = comments['sender_id'][i]
    response = requests.get('https://api.vk.com/method/users.get',
        params={'access_token': TOKEN_USER,
        'v': VERSION,
        'user_ids' : user_id,
        'fields' : fields
        })
    try:
        data3 = response.json()['response']
    except:
        response_error_counter +=1
        continue
    for elem in data3:
        id_4 = elem['id']
        people.loc[id_4, 'id'] = elem['id']
        people.loc[id_4, 'first_name'] = elem['first_name']
        people.loc[id_4, 'last_name'] = elem['last_name']
        people.loc[id_4, 'sex'] = elem['sex']
        try:
            people.loc[id_4, 'city'] = elem['city']['title']
            people.loc[id_4, 'b_date'] = elem['b_date']
            people.loc[id_4, 'relation'] = elem['relation']
            people.loc[id_4, 'university'] = elem['university_name']
            people.loc[id_4, 'career'] = elem['career']
            people.loc[id_4, 'faculty'] = elem['faculty_name']
        except:
            continue

In [ ]:
people = people.drop(columns = ['index', 'faculty', 'career', 'b_date'])

### Сохранение

In [174]:
people.to_csv(path_users)
comments.to_csv(path_com)
df.to_csv(path)